In [ ]:
# https://stackoverflow.com/questions/66116155/how-to-tell-pytorch-which-cuda-version-to-take
# https://github.com/pytorch/pytorch/issues/75992

# Previous Versions of Pytorch avaiable here: https://pytorch.org/get-started/previous-versions/
#! pip install --upgrade torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
#! pip install --upgrade torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
#! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 #<--This version works for CUDA 11.4



In [ ]:
! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 


In [2]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [3]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [4]:
torch.__version__
#'1.12.0+cu111'
#'1.9.0+cu111' This is the version by default


'1.12.1+cu113'

In [5]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [6]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device
device = 'cuda'


In [ ]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

In [ ]:
x.get_device()

In [ ]:
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

In [ ]:
torch.cuda.empty_cache()

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @xxxxxp
>

In [8]:
#Available Datasets

def params(): 
    
    code_models = {
        'Case1':('EleutherAI/gpt-neo-125M','codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv','EleutherAI-gpt-neo-125M_10000_','callbacks-EleutherAI-gpt-neo-125M_10000_'),
        'Case2':('EleutherAI/gpt-neo-2.7B','codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv','EleutherAI-gpt-neo-2.7B_10000_','callbacks-EleutherAI-gpt-neo-2.7B_10000_'),    
        'Case3':('EleutherAI/gpt-neo-1.3B','codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000.csv','EleutherAI-gpt-neo-1.3B_10000_','callbacks-EleutherAI-gpt-neo-1.3B_10000_'),
        'Case4':('microsoft/CodeGPT-small-py','codesearch_tesbed_microsoft-CodeGPT-small-py_1024_10000.csv','CodeGPT-small-py_10000_','callbacks-CodeGPT-small-py_10000_'),
        'Case5':('microsoft/CodeGPT-small-py-adaptedGPT2','codesearch_tesbed_microsoft-CodeGPT-small-py-adaptedGPT2_1024_10000.csv','CodeGPT-small-py-adaptedGPT2_10000_','callbacks-CodeGPT-small-py-adaptedGPT2_10000_'),
        'Case6':('Salesforce/codegen-2B-multi','codesearch_tesbed_Salesforce-codegen-2B-multi_1024_10000.csv','Salesforce-codegen-2B-multi_1024_10000_','callbacks-Salesforce-codegen-2B-multi_1024_10000_'),
        'Case7':('eleutherai/gpt-neox-20b','codesearch_tesbed_EleutherAI-gpt-neox-20b_1024_10000.csv','EleutherAI-gpt-neoX-20b_1024_10000_','callbacks-EleutherAI-gpt-neoX_1024_10000_')
    }
    current_case = 'Case7' #<----[Hyper]
    
    #print(code_models[current_case][1])
    
    return {
            'big_table_path' : '../data/concept_tables/' + code_models[current_case][1],
            'hf_model' :  code_models[current_case][0],
            'model_name': code_models[current_case][2],
            'callbacks' : '../datax/' + code_models[current_case][3],
            'wpe':2048#1024  #<----[Hyper]
}

In [9]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neox-20b_1024_10000.csv'

# Data Upload

In [10]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [11]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,247.111600
std,191.178002
min,34.000000
25%,111.000000
50%,184.000000
75%,321.000000
max,1024.000000


In [12]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def auth(self, skypeToken):\n """"""\n ...","[('def', 'def', 'function_definition'), ('auth...","[(1545, 'def', 'function_definition'), (24896,...","[1545, 24896, 9, 1286, 13, 1629, 1692, 11200, ...",711
1,"def template_tag(library, name):\n """"""\n ...","[('def', 'def', 'function_definition'), ('temp...","[(1545, 'def', 'function_definition'), (7646, ...","[1545, 7646, 64, 7784, 9, 17921, 13, 1416, 226...",223
2,"def _search_tree_backwards(self, template, par...","[('def', 'def', 'function_definition'), ('_sea...","[(1545, 'def', 'function_definition'), (795, '...","[1545, 795, 8716, 64, 12588, 64, 2135, 4515, 9...",181
3,"def get_upcoming_events(self):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(1545, 'def', 'function_definition'), (755, '...","[1545, 755, 64, 484, 4202, 64, 20969, 9, 1286,...",274
4,"def get_markup( self, tag_name ):\n """"...","[('def', 'def', 'function_definition'), ('get_...","[(1545, 'def', 'function_definition'), (50276,...","[1545, 50276, 788, 64, 4698, 484, 9, 1881, 13,...",63


In [13]:
data_pd.iat[243, 1]

'[(\'def\', \'def\', \'function_definition\'), (\'set_server_admin_password\', \'identifier\', \'function_definition\'), (\'(\', \'(\', \'parameters\'), (\'self\', \'identifier\', \'parameters\'), (\',\', \',\', \'parameters\'), (\'server_name\', \'identifier\', \'parameters\'), (\',\', \',\', \'parameters\'), (\'admin_password\', \'identifier\', \'parameters\'), (\')\', \')\', \'parameters\'), (\':\', \':\', \'function_definition\'), ("\'\'\'\\n        Reset the administrator password for a server.\\n\\n        server_name:\\n            Name of the server to change the password.\\n        admin_password:\\n            The new administrator password for the server.\\n        \'\'\'", \'string\', \'expression_statement\'), (\'_validate_not_none\', \'identifier\', \'call\'), (\'(\', \'(\', \'argument_list\'), ("\'server_name\'", \'string\', \'argument_list\'), (\',\', \',\', \'argument_list\'), (\'server_name\', \'identifier\', \'argument_list\'), (\')\', \')\', \'argument_list\'), (\'_

# Model Upload

In [14]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git

def testing_1():
    #! pip install transformers
    from transformers import AutoTokenizer, AutoModelForCausalLM
    from transformers import pipeline
    # This is for 'EleutherAI/gpt-neo-125M'
    tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
    generator = pipeline(
        'text-generation', 
        model= parameters['hf_model'] )
    
    #TEST: Example 1: generation
    generator(
        "EleutherAI has", 
        do_sample=True, 
        max_new_tokens=20, 
        pad_token_id = tokenizer.eos_token_id
        )
    
    #TEST: Example 2: generation

    generator( 
        data_pd.whole_func_string.values[0][:100], #<-- Code data
        do_sample=True, 
        max_new_tokens=20,
        pad_token_id = tokenizer.eos_token_id 
        )



# Extracting Logits From a Given Model

In [ ]:
#This code works for GPT-Neo
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'], ignore_mismatched_sizes=True )
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [15]:
parameters['hf_model']

'eleutherai/gpt-neox-20b'

In [17]:
#This code works for CodeGPT, Salesforce and GPTNeoX
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
model = AutoModelForCausalLM.from_pretrained(parameters['hf_model'],device_map="auto", torch_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
model.to( device )

In [ ]:
#tokenizer.get_vocab()
#len( tokenizer.get_vocab() ) #todo an assert

In [18]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [19]:
len(prompts)

10000

In [20]:
#Making space for the tensors
#input_ids_list = tokenizer.batch_encode_plus( prompts ) #<-- Do not return as a Tensor
input_ids_list = tokenizer.batch_encode_plus( list(prompts) ) #<-- Do not return as a Tensor [cast to list]
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [21]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]

In [22]:
input_ids_list[0]

tensor([ 1545, 24896,     9,  1286,    13,  1629,  1692, 11200,  2262,   187,
        50270, 32488,   187, 50270,  6825,   247,   747, 12960, 10669,   970,
          247,  1655, 45385, 10669,    15,   535, 50270, 16523,    27,   187,
        50266,  3319,  1692, 11200,   313,  1344,  2262,  5368, 45385, 10669,
          535, 50270, 34551,    27,   187, 50266,     9,  1344,    13, 36743,
           15, 36377,    13,  1213,    13, 45385, 36997,    10, 31343,    27,
        12960, 10669,    13,  2330,   866, 11028,   604,  1929,    13,   187,
        50254, 50254, 50262,  6870,   272, 21229,  3167,  1590,    13, 21229,
          604,  2530,   535, 50270, 21328,  3013,    27,   187, 50266,    15,
        15992,  1692, 15714,  5330,    27,   604,   253, 16164,  2748,   310,
        10945,   187, 50266,    15, 15992,  1692, 22444,  5330,    27,   604,
          253, 16164,   830,   476,   626,   320, 11742,   187, 50270, 32488,
          187, 50270, 13763,   426,   866, 11028,   426, 21229, 

In [23]:
#It should be same size
print(len(input_ids_list),len(prompts))
assert len(input_ids_list) == len(prompts)

10000 10000


In [24]:
data_pd.model_input_ids.values[0]

'[1545, 24896, 9, 1286, 13, 1629, 1692, 11200, 2262, 187, 50270, 32488, 187, 50270, 6825, 247, 747, 12960, 10669, 970, 247, 1655, 45385, 10669, 15, 535, 50270, 16523, 27, 187, 50266, 3319, 1692, 11200, 313, 1344, 2262, 5368, 45385, 10669, 535, 50270, 34551, 27, 187, 50266, 9, 1344, 13, 36743, 15, 36377, 13, 1213, 13, 45385, 36997, 10, 31343, 27, 12960, 10669, 13, 2330, 866, 11028, 604, 1929, 13, 187, 50254, 50254, 50262, 6870, 272, 21229, 3167, 1590, 13, 21229, 604, 2530, 535, 50270, 21328, 3013, 27, 187, 50266, 15, 15992, 1692, 15714, 5330, 27, 604, 253, 16164, 2748, 310, 10945, 187, 50266, 15, 15992, 1692, 22444, 5330, 27, 604, 253, 16164, 830, 476, 626, 320, 11742, 187, 50270, 32488, 187, 50270, 13763, 426, 866, 11028, 426, 21229, 426, 8256, 187, 50270, 4856, 84, 15310, 426, 45385, 13457, 15, 11252, 64, 24691, 5648, 13792, 187, 50270, 6050, 417, 10669, 27, 187, 50266, 1704, 84, 426, 540, 9, 2606, 15, 2606, 6649, 187, 50266, 13362, 426, 1881, 15, 788, 13815, 9726, 16850, 9, 1344, 9, 

In [25]:
input_ids_list[0]

tensor([ 1545, 24896,     9,  1286,    13,  1629,  1692, 11200,  2262,   187,
        50270, 32488,   187, 50270,  6825,   247,   747, 12960, 10669,   970,
          247,  1655, 45385, 10669,    15,   535, 50270, 16523,    27,   187,
        50266,  3319,  1692, 11200,   313,  1344,  2262,  5368, 45385, 10669,
          535, 50270, 34551,    27,   187, 50266,     9,  1344,    13, 36743,
           15, 36377,    13,  1213,    13, 45385, 36997,    10, 31343,    27,
        12960, 10669,    13,  2330,   866, 11028,   604,  1929,    13,   187,
        50254, 50254, 50262,  6870,   272, 21229,  3167,  1590,    13, 21229,
          604,  2530,   535, 50270, 21328,  3013,    27,   187, 50266,    15,
        15992,  1692, 15714,  5330,    27,   604,   253, 16164,  2748,   310,
        10945,   187, 50266,    15, 15992,  1692, 22444,  5330,    27,   604,
          253, 16164,   830,   476,   626,   320, 11742,   187, 50270, 32488,
          187, 50270, 13763,   426,   866, 11028,   426, 21229, 

In [26]:
# tokenizer.decode( input_ids_list[0] ) #Decoding IDs

In [27]:
tokenizer.vocab_size #Check this value with WTE in the model

50254

In [ ]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

In [28]:
parameters #Verification Point of Parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neox-20b_1024_10000.csv',
 'hf_model': 'eleutherai/gpt-neox-20b',
 'model_name': 'EleutherAI-gpt-neoX-20b_1024_10000_',
 'callbacks': '../datax/callbacks-EleutherAI-gpt-neoX_1024_10000_',
 'wpe': 2048}

In [29]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        
        #output = [ model( 
        #    input_ids = i, 
        #    labels = i.type(torch.LongTensor).to(device) 
        #    ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
        output = []
        for i in input[n:n+batch]:
            #print(i)
            output.append( 
                model( 
                    input_ids = i, 
                    labels = i.to(torch.long).to(device) 
                )
            )
        
        #print(output)
        
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        ###Acuracy
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).
        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{current_batch}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy', output_loss)
        
        print(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:         
            del out
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
        del output
        del output_logits
        del output_loss
            

    pass

In [30]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [31]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 1, input= input_ids_list, from_index=0)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│ in logit_extractor                                                                               │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/accelerate/hooks.py:165 in new_forward                    │
│                                                                                                  │
│   162 │   │   │   with torch.no_grad():                                                          │
│   163 │   │   │   │   output = old_forward(*args, **kwargs)                                      │
│   164 │   │   else:                                                                              │
│ ❱ 165 │   │   │   output = old_forward(*args, **kwargs)                                          │
│   166 │   │   return module._hf_hook.post_forward(module, output)                                │
│   167 │                                                                                          │
│   168 │   module.forward = new_forward                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:654 in  │
│ forward                                                                                          │
│                                                                                                  │
│   651 │   │   ```"""                                                                             │
│   652 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return   │
│   653 │   │                                                                                      │
│ ❱ 654 │   │   outputs = self.gpt_neox(                                                           │
│   655 │   │   │   input_ids,                                                                     │
│   656 │   │   │   attention_mask=attention_mask,                                                 │
│   657 │   │   │   head_mask=head_mask,                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_

In [ ]:

#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [ ]:

torch.cuda.empty_cache()



In [ ]:
input_ids_list[245].shape



In [ ]:
i = torch.rand(1000)
i = i.to(int)
i.shape
#i.type(torch.LongTensor).to(device)

In [ ]:
del i 
del model

In [ ]:
model( input_ids = i, labels = i.to(torch.long).to(device))


In [ ]:
del model
gc.collect()

In [ ]:
i =input_ids_list[243]

In [ ]:
i

In [ ]:
i.to(torch.long)

In [ ]:
i.to(device)

In [ ]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [ ]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [ ]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [ ]:
#output_loss